<a href="https://colab.research.google.com/github/YG15/DataHack2019/blob/master/yonathan_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DataHack 2019 Armis project**

In [0]:
import pandas as pd
import numpy as np

In [0]:
!wget https://raw.githubusercontent.com/YG15/DataHack2019/master/src/functions/get_network_hierarchy.py

from get_network_hierarchy import *

--2019-09-02 20:30:23--  https://raw.githubusercontent.com/YG15/DataHack2019/master/src/functions/get_network_hierarchy.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1703 (1.7K) [text/plain]
Saving to: ‘get_network_hierarchy.py’

get_network_hierarc 100%[===================>]   1.66K  --.-KB/s    in 0s      

2019-09-02 20:30:23 (234 MB/s) - ‘get_network_hierarchy.py’ saved [1703/1703]



In [0]:
url = 'https://raw.githubusercontent.com/YG15/DataHack2019/master/data/draft/network_draft.csv'
df = pd.read_csv(url)
df.head()

,network_id,device_id,service_device_id,timestamp,real_hierarchy
0,1,47.0,932,1564963200,2
1,1,440.0,12862,1564963200,1
2,1,440.0,47,1564963200,1
3,1,440.0,131,1564963200,1
4,0,52.0,430,1564963200,1


In [0]:
get_network_hierarchy(df).head()

,network_id,device_id,service_device_id,timestamp,real_hierarchy,full_service_device_id,full_device_id,network_hierarchy
0,1,47.0,932,1564963200,2,1_932,1_47,2.0
1,1,440.0,12862,1564963200,1,1_12862,1_440,1.0
2,1,440.0,47,1564963200,1,1_47,1_440,1.0
3,1,440.0,131,1564963200,1,1_131,1_440,1.0
4,0,52.0,430,1564963200,1,0_430,0_52,1.0


In [0]:
def get_network_hierarchy(network_df):
  """
  input: a table of network connection
  output: the same table with additional columns representing the level of hierarchy in the net
  IPORTANT: the function assumes there each device has it unique id regardless the network it is in, if there are duplicates ids, the function will results incorrect results
            thus it is recommended to run the function on each network seperatly or to change the devices ids so they won't be any shared ids across networks
  """
  # Correct columns values from float tp integers
  network_df['full_service_device_id'] = [str(int(network))+'_'+str(int(device)) if ~np.isnan(device) else None\
                                          for network,device in zip(network_df.network_id,network_df.service_device_id)]
  network_df['full_device_id']         = [str(int(network))+'_'+str(int(device)) if ~np.isnan(device) else None\
                                          for network,device in zip(network_df.network_id,network_df.device_id)]
  
  output_devices = list(network_df.full_device_id.unique())
  input_devices  = list(network_df.full_service_device_id.unique())
  root_devices   = list(set(output_devices).difference(set(input_devices)))
  
  device_level = 0
  network_df['network_hierarchy'] = np.nan
  
  while len(root_devices)>0:
    network_df['network_hierarchy'] = [device_level+1 if device in root_devices else val for device, val in zip(network_df.full_device_id,network_df.network_hierarchy)]
    root_devices = list(network_df.full_service_device_id[network_df.network_hierarchy==device_level+1])
    device_level+=1
  return(network_df)